<a href="https://colab.research.google.com/github/jimmyshah83/deep_learning/blob/master/GAN_Keras.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

GAN Using Keras

In [1]:
#from google.colab import drive
#drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
#import zipfile
#import io
#from zipfile import ZipFile
#with ZipFile('drive/My Drive/cropped.zip', 'r') as zf:
#   zf.extractall('drive/My Drive/images/')

In [0]:
# Hyperparameters
IMAGE_SIZE = 64
NOISE_SIZE = 100
LR_D = 0.00004
LR_G = 0.0004
BATCH_SIZE = 256
EPOCHS = 1
BETA1 = 0.5
WEIGHT_INIT_STDDEV = 0.02
EPSILON = 0.00005
SAMPLES_TO_SHOW = 5

In [4]:
import os
import time
import numpy as np
from glob import glob
import datetime
import random
from PIL import Image
import matplotlib.pyplot as plt
from __future__ import absolute_import, division, print_function

import tensorflow as tf
from keras.models import Model, Sequential
from keras.layers import *
from keras.optimizers import Adam
from keras.initializers import TruncatedNormal

%matplotlib inline

Using TensorFlow backend.


In [0]:
train_images = np.asarray([np.asarray(Image.open(file).resize((IMAGE_SIZE, IMAGE_SIZE))) 
                           for file in glob('drive/My Drive/images/*.png')])

print ("Input: " + str(train_images.shape))

In [0]:
train_images = (train_images - 127.5) / 127.5 # Normalize the images to [-1, 1]
# Batch and shuffle the data
train_dataset = tf.data.Dataset.from_tensor_slices(train_images).shuffle(9877).batch(BATCH_SIZE)

In [0]:
train_dataset

In [0]:
def make_generator_model():
    model = Sequential()
    
    model.add(Dense(8*8*1024, use_bias=False, input_shape=(100,)))
    model.add(Reshape((8, 8, 1024)))
    model.add(LeakyReLU())
    
    model.add(Conv2DTranspose(512, (5, 5), padding='same', strides=2, kernel_initializer=TruncatedNormal(stddev=WEIGHT_INIT_STDDEV)))
    model.add(BatchNormalization(epsilon=EPSILON))
    model.add(LeakyReLU())
    
    
    model.add(Conv2DTranspose(256, (5, 5), padding='same', strides=2, kernel_initializer=TruncatedNormal(stddev=WEIGHT_INIT_STDDEV)))
    model.add(BatchNormalization(epsilon=EPSILON))
    model.add(LeakyReLU())
    
    model.add(Conv2DTranspose(128, (5, 5), padding='same', strides=2, kernel_initializer=TruncatedNormal(stddev=WEIGHT_INIT_STDDEV)))
    model.add(BatchNormalization(epsilon=EPSILON))
    model.add(LeakyReLU())
    
    model.add(Conv2DTranspose(64, (5, 5), padding='same', strides=2, kernel_initializer=TruncatedNormal(stddev=WEIGHT_INIT_STDDEV)))
    model.add(BatchNormalization(epsilon=EPSILON))
    model.add(LeakyReLU())
    
    model.add(Conv2DTranspose(1, (5, 5), padding='same', strides=2, kernel_initializer=TruncatedNormal(stddev=WEIGHT_INIT_STDDEV)))
    
    return model

In [0]:
generator = make_generator_model()

noise = tf.random.normal([1, 100])
generated_image = generator(noise, training=False)

plt.imshow(generated_image[0, :, :, 0], cmap='gray')

In [0]:
def make_discriminator_model():
  model = Sequential()
  model.add(Conv2D(64, (5, 5), padding='same', strides=2, kernel_initializer=TruncatedNormal(stddev=WEIGHT_INIT_STDDEV)))
  model.add(BatchNormalization(epsilon=EPSILON))
  model.add(LeakyReLU())
    
  model.add(Conv2D(128, (5, 5), padding='same', strides=2, kernel_initializer=TruncatedNormal(stddev=WEIGHT_INIT_STDDEV)))
  model.add(BatchNormalization(epsilon=EPSILON))
  model.add(LeakyReLU())
    
  model.add(Conv2D(256, (5, 5), padding='same', strides=2, kernel_initializer=TruncatedNormal(stddev=WEIGHT_INIT_STDDEV)))
  model.add(BatchNormalization(epsilon=EPSILON))
  model.add(LeakyReLU())
              
  model.add(Conv2D(514, (5, 5), padding='same', strides=2, kernel_initializer=TruncatedNormal(stddev=WEIGHT_INIT_STDDEV)))
  model.add(BatchNormalization(epsilon=EPSILON))
  model.add(LeakyReLU())
              
  model.add(Conv2D(1024, (5, 5), padding='same', strides=2, kernel_initializer=TruncatedNormal(stddev=WEIGHT_INIT_STDDEV)))
  model.add(BatchNormalization(epsilon=EPSILON))
  model.add(LeakyReLU())
              
  model.add(Reshape((8, 8, 1024), input_shape=(8*8*1024,)))
  model.add(Dense(1))
  model.add(Activation('sigmoid'))
              
  return model

In [0]:
discriminator = make_discriminator_model()

In [0]:
cross_entropy = tf.keras.losses.BinaryCrossentropy(from_logits=True)

In [0]:
def discriminator_loss(real_output, fake_output):
    real_loss = cross_entropy(tf.ones_like(real_output), real_output)
    fake_loss = cross_entropy(tf.zeros_like(fake_output), fake_output)
    total_loss = real_loss + fake_loss
    return total_loss

In [0]:
def generator_loss(fake_output):
    return cross_entropy(tf.ones_like(fake_output), fake_output)

In [0]:
discriminator_optimizer = Adam(lr=LR_D, beta_1=BETA1)
generator_optimizer = Adam(lr=LR_G, beta_1=BETA1)

In [0]:
def train_step(images):
    noise = tf.random.normal([BATCH_SIZE, NOISE_SIZE])

    with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
      generated_images = generator(noise, training=True)

      real_output = discriminator(images, training=True)
      fake_output = discriminator(generated_images, training=True)

      gen_loss = generator_loss(fake_output)
      disc_loss = discriminator_loss(real_output, fake_output)

    gradients_of_generator = gen_tape.gradient(gen_loss, generator.trainable_variables)
    gradients_of_discriminator = disc_tape.gradient(disc_loss, discriminator.trainable_variables)

    generator_optimizer.apply_gradients(zip(gradients_of_generator, generator.trainable_variables))
    discriminator_optimizer.apply_gradients(zip(gradients_of_discriminator, discriminator.trainable_variables))

In [0]:
def train(dataset):  
  for epoch in range(EPOCHS):
    start = time.time()
    
    for image_batch in dataset:
      train_step(image_batch)
    
    print ('Time for epoch {} is {} sec'.format(epoch + 1, time.time()-start))

In [0]:
train(train_dataset)